The US National Oceanic and Atmospheric Administration (NOAA) publishes climate data through the National Centers for Environmental Information (NCEI), with daily measurements called the [Global \[Surface\] Summary of Day (GSOD)](https://www.ncei.noaa.gov/access/metadata/landing-page/bin/iso?id=gov.noaa.ncdc:C00516). Per this page, "The data are reported and summarized based on Greenwich Mean Time (GMT, 0000Z - 2359Z) since the original synoptic/hourly data are reported and based on GMT." 

The data is available in many forms, including raw CSV [here](https://www.ncei.noaa.gov/data/global-summary-of-the-day/access/). This is organized by year, and then by a file per data-collecting station. The stations are identified by a combination of USAF (Air Force station ID) and WBAN (NCDC (National Climatic Data Center) Weather Bureau Army Navy number) identifiers. These stations are further located by latitude and longitude in this stations list file: [https://www.ncei.noaa.gov/pub/data/noaa/isd-history.csv](https://www.ncei.noaa.gov/pub/data/noaa/isd-history.csv) (and in [txt](https://www.ncei.noaa.gov/pub/data/noaa/isd-history.txt) format)

I want to do analysis on climate data for Chicago year over year, so I want to find a station in the Chicago area with the greatest year-over-year coverage of data (maximum days per year of data). 

Taking the latitude/longitude of the approximate center of Chicago per [OpenStreetMap](https://wiki.openstreetmap.org/wiki/Chicago,_Illinois#:~:text=Chicago%20is%20a%20city%20in,%C2%B039%E2%80%B254.00%E2%80%B3%20West.) as `(41.868, -87.665)`, we can find the closest station using the [Haversine formula](https://en.wikipedia.org/wiki/Haversine_formula) for great circle distances. 

In [73]:
import pandas as pd
from haversine import haversine
import requests
import datetime

def distanceToChicago(dfRow):
    chicagoLatLong = (41.868, -87.665)
    return haversine((dfRow['LAT'], dfRow['LON']), chicagoLatLong)

stations = pd.read_csv('https://www.ncei.noaa.gov/pub/data/noaa/isd-history.csv', dtype={'BEGIN': 'string', 'END': 'string'})
# stations = pd.read_csv('https://www.ncei.noaa.gov/pub/data/noaa/isd-history.csv')
stations.dropna(subset=['LAT', 'LON'], inplace=True)  
stations['distance_to_chicago'] = stations.apply(distanceToChicago, axis=1)
stations.sort_values('distance_to_chicago', inplace=True)

print(stations.head(10))

         USAF   WBAN                      STATION NAME CTRY STATE  ICAO  \
29584  999999  94866               CHICAGO MEIGS FIELD   US    IL   CGX   
28239  998499  99999                  NORTHERLY ISLAND   US    IL   NaN   
20066  725346  94866                     CHICAGO/MEIGS   US    IL  KCGX   
20067  725346  99999                     CHICAGO/MEIGS   US    IL  KCGX   
28732  999999  14819                 CHICAGO MIDWAY AP   US    IL  KMDW   
20060  725340  14819          CHICAGO MIDWAY INTL ARPT   US    IL  KMDW   
28238  998497  99999                FOSTER AVE CHICAGO   US    IL   NaN   
21613  744650  99999                    DUNNE CRIB IL.   US   NaN   NaN   
27689  997255  99999             9087044 - CALUMET  IL   US   NaN   NaN   
20063  725344  99999  CHICAGO / CALUMET COAST GUARD ST   US   NaN   NaN   

          LAT     LON  ELEV(M)     BEGIN       END  distance_to_chicago  
29584  41.867 -87.617    180.1  19660101  19700701             3.976238  
28239  41.856 -87.609    1

In the above list, the first 10 stations are within 20 kilometers of Chicago. However, looking at their "END" dates I can see that many aren't up to the current day. I'm only interested in stations that have current data. 

In [75]:
todayYear = str(datetime.date.today().year)
stationsCurrent = stations[stations['END'].str.startswith(todayYear)]
print(stationsCurrent.head(10))

         USAF   WBAN                          STATION NAME CTRY STATE  ICAO  \
28239  998499  99999                      NORTHERLY ISLAND   US    IL   NaN   
20060  725340  14819              CHICAGO MIDWAY INTL ARPT   US    IL  KMDW   
27689  997255  99999                 9087044 - CALUMET  IL   US   NaN   NaN   
27765  997338  99999                               CHICAGO   US    IL   NaN   
20027  725300  94846  CHICAGO O'HARE INTERNATIONAL AIRPORT   US    IL  KORD   
21634  744665   4838              PALWAUKEE MUNICIPAL ARPT   US    IL  KPWK   
20057  725337   4807                  GARY/CHICAGO AIRPORT   US    IN  KGYY   
18063  722126   4879                  LANSING MUNICIPAL AP   US    IL  KIGQ   
20070  725348   4831              LEWIS UNIVERSITY AIRPORT   US    IL  KLOT   
20028  725305  94892                        DUPAGE AIRPORT   US    IL  KDPA   

          LAT     LON  ELEV(M)     BEGIN       END  distance_to_chicago  
28239  41.856 -87.609    190.0  20120618  20231121      

Visually inspecting this, I see the two stations with the highest date coverage (at least, according to this index file) and the closest distance to Chicago are "CHICAGO MIDWAY INTL ARPT" and "CHICAGO O'HARE INTERNATIONAL AIRPORT". This makes sense at first glance, given the climatological impact on the business of air travel. 

Later we'll need to confirm quality and coverage of data with the actual data files.

In [76]:
chicagoStations = stations[stations['STATION NAME'].isin(['CHICAGO MIDWAY INTL ARPT', 'CHICAGO O\'HARE INTERNATIONAL AIRPORT'])]
print(chicagoStations)

         USAF   WBAN                          STATION NAME CTRY STATE  ICAO  \
20060  725340  14819              CHICAGO MIDWAY INTL ARPT   US    IL  KMDW   
20027  725300  94846  CHICAGO O'HARE INTERNATIONAL AIRPORT   US    IL  KORD   

          LAT     LON  ELEV(M)     BEGIN       END  distance_to_chicago  
20060  41.784 -87.755    185.8  19730101  20231122             11.95220  
20027  41.960 -87.932    204.8  19461001  20231122             24.34657  


How many days of actual data should we expect per station? This would be the number of days between the BEGIN and END dates, inclusive.

In [77]:
def daysExpected(station):
    begin = datetime.datetime.strptime(station['BEGIN'], '%Y%m%d')
    end = datetime.datetime.strptime(station['END'], '%Y%m%d')
    return (end - begin).days

for station in chicagoStations.iloc:
    print(f"{station['STATION NAME']}: {daysExpected(station)} days")

CHICAGO MIDWAY INTL ARPT: 18587 days
CHICAGO O'HARE INTERNATIONAL AIRPORT: 28176 days


In [92]:
def checkUrlExists(url):
    try:
        # By only sending an HTTP HEAD request, we don't consume bandwidth 
        # by actually requesting the whole resource. We just check that it responds.
        response = requests.head(url)
        return response.status_code  == requests.codes.ok
    except requests.ConnectionError:
        print(f"Failed to connect to {url}.")
        return False

def gsodUrl(station, year):
    usaf, wban = station['USAF'], station['WBAN']
    return f"https://www.ncei.noaa.gov/data/global-summary-of-the-day/access/{year}/{usaf}{wban}.csv"

def checkAllDataExists(station):
    startYear, endYear = station['BEGIN'][:4], station['END'][:4]
    allExists = True
    for year in range(int(startYear), int(endYear) + 1):
        requestUrl = gsodUrl(station, year)
        if checkUrlExists(requestUrl):
            print(f"Data exists for year {year} at {requestUrl}")
        else:
            print(f"WARNING: Missing data for year {year} at {requestUrl}")
            allExists = False
    return allExists

# Let's check each of the Chicago stations to make sure we have all the data we expect:
for station in chicagoStations.iloc:
    name = station['STATION NAME']
    print(f"Checking that data exists for {name}")
    checkAllDataExists(station)

Checking that data exists for CHICAGO MIDWAY INTL ARPT
Data exists for year 1973 at https://www.ncei.noaa.gov/data/global-summary-of-the-day/access/1973/72534014819.csv
Data exists for year 1974 at https://www.ncei.noaa.gov/data/global-summary-of-the-day/access/1974/72534014819.csv
Data exists for year 1975 at https://www.ncei.noaa.gov/data/global-summary-of-the-day/access/1975/72534014819.csv
Data exists for year 1976 at https://www.ncei.noaa.gov/data/global-summary-of-the-day/access/1976/72534014819.csv
Data exists for year 1977 at https://www.ncei.noaa.gov/data/global-summary-of-the-day/access/1977/72534014819.csv
Data exists for year 1978 at https://www.ncei.noaa.gov/data/global-summary-of-the-day/access/1978/72534014819.csv
Data exists for year 1979 at https://www.ncei.noaa.gov/data/global-summary-of-the-day/access/1979/72534014819.csv
Data exists for year 1980 at https://www.ncei.noaa.gov/data/global-summary-of-the-day/access/1980/72534014819.csv
Data exists for year 1981 at http

Oh no! There are missing data files for years 1970, 1971, and 1972 for the "CHICAGO O'HARE INTERNATIONAL AIRPORT" station. This won't do. Interestingly, this data is available when going through [this page](https://www.ncdc.noaa.gov/cdo-web/datasets/GHCND/stations/GHCND:USW00094846/detail), but the files apparently aren't present in this GSOD CSV format. 

I'll just use the "CHICAGO MIDWAY INTL ARPT" station then and satisfy myself with data from 1973 to present day. Based on the BEGIN/END dates in the original station index file, I'm expecting to see 18587 days of data whereas the actual data we get back from GSOD has 18546 days worth of measurements, leaving a discreprancy of 41 days. The cause of this discrepancy may be in how the Python `timedelta` object calculates difference of two dates ([https://docs.python.org/3/library/datetime.html#timedelta-objects](https://docs.python.org/3/library/datetime.html#timedelta-objects)).  

In [104]:
def getAllData(station):
    startYear, endYear = station['BEGIN'][:4], station['END'][:4]
    name = station['STATION NAME']
    data = []
    print(f"Downloading data for {name} from {startYear} to {endYear}")
    for year in range(int(startYear), int(endYear) + 1):
        requestUrl = gsodUrl(station, year)
        print(f"Downloading data for year {year} ({requestUrl})")
        data.append(pd.read_csv(gsodUrl(station, year)))
    return pd.concat(data)
    
station = stations[stations['STATION NAME'] == 'CHICAGO MIDWAY INTL ARPT'].iloc[0]
gsod = getAllData(station)
print(gsod.describe())

            STATION      LATITUDE     LONGITUDE     ELEVATION          TEMP  \
count  1.854600e+04  18546.000000  18546.000000  18546.000000  18546.000000   
mean   7.253401e+10     41.785958    -87.752444    186.446366     51.634083   
std    0.000000e+00      0.000529      0.000777      0.186197     19.890515   
min    7.253401e+10     41.784120    -87.755140    185.800000    -17.900000   
25%    7.253401e+10     41.786110    -87.752220    186.500000     35.900000   
50%    7.253401e+10     41.786110    -87.752220    186.500000     52.600000   
75%    7.253401e+10     41.786110    -87.752220    186.500000     69.300000   
max    7.253401e+10     41.786110    -87.752220    186.500000     94.000000   

       TEMP_ATTRIBUTES          DEWP  DEWP_ATTRIBUTES           SLP  \
count     18546.000000  18546.000000     18546.000000  18546.000000   
mean         23.449800     53.370047        23.345034   4019.329909   
std           1.285667    365.920106         1.762054   4237.744746   
min 

Now that I've selected the station and downloaded all historical GSOD data, I'll export to CSV for manual import into Google BigQuery for further querying and analysis in Tableau Public.

In [105]:
gsod.to_csv(f"./GSOD - {station['STATION NAME']} - {station['USAF']}{station['WBAN']}.csv")